In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
char_set = list(set(sentence))
char_dict = {c: i for i,c in enumerate(char_set)}

In [4]:
char_dict

{' ': 0,
 'a': 1,
 'm': 2,
 'n': 3,
 'l': 4,
 'e': 5,
 '.': 6,
 'o': 7,
 'f': 8,
 'y': 9,
 'p': 10,
 "'": 11,
 'u': 12,
 'i': 13,
 ',': 14,
 'b': 15,
 's': 16,
 'k': 17,
 'c': 18,
 't': 19,
 'w': 20,
 'g': 21,
 'd': 22,
 'h': 23,
 'r': 24}

In [5]:
dict_size = len(char_dict)

In [6]:
hidden_size = dict_size
sequence_length = 10
learning_rate = 0.1

In [ ]:
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i+sequence_length]
    print(f"{i}번째 x_str: {x_str}")
    y_str = sentence[i+1: i + sequence_length + 1]
    print(f"{i}번째 y_str: {y_str}")

    x_data.append([char_dict[c] for c in x_str])
    y_data.append([char_dict[c] for c in y_str])
            

In [8]:
print(x_data[0], y_data[0])

[13, 8, 0, 9, 7, 12, 0, 20, 1, 3] [8, 0, 9, 7, 12, 0, 20, 1, 3, 19]


In [12]:
x_one_hot = [np.eye(dict_size)[x] for x in x_data]

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

C:\Users\dltkd\AppData\Local\Temp\ipykernel_10528\148319804.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  X = torch.FloatTensor(x_one_hot)


In [15]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias=True)
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [16]:
net = Net(dict_size, hidden_size, 2)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [18]:
outputs = net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


In [20]:
print(outputs.reshape(-1, dict_size).shape)
print(Y.reshape(-1).shape)

torch.Size([1700, 25])
torch.Size([1700])


In [22]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.reshape(-1, dict_size), Y.reshape(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j==0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]

    print(predict_str)

ssinyeeeerieiieieyreyrerresiyeyeneieoeeeeeiseieeeeieeeireieiieseeeeriereyeirreieyeneireeyeeeireeeireeeireeieyreeineiineereneieeereireeeiieneereeiieireieeereyreseeeoreieeieiireieee
 tt   t    t        t             t    t  t                          t    t   t     t      t    t         t         t                 t       t                                    
 nt meeon  ys  yst nys yyne ysnmnnsomyn ns nyn nn nymnys mys  yse nyt ynmmnse yse nyt myyn nys nyse yytemlyss nyt nyyns nnnysnmnst nmnyse yne nye nyt myt nns yse nsenyynyseys mys 
                                                                                             e                    e                                                                
 ttooctwortwooccwcoctpococo opoctwopopowoor  ooporocorwooctrocororrrororcroorct octworopt oopor toor  wor wooctcopoctroco ropocoropocctooct otrorct ort oororoccprwrocococororororr
lttosttoostto'nitrtttossittttosttooosiitostgtrttrtostiossitostitettrttossioosttoslttoossitrtossttoss